# AI Saturdays Training Exercise - Bank Classifier
Las campañas de marketing del banco dependen de los datos de los clientes. El tamaño de estos datos es tan grande que es imposible que un analista de datos extraiga buena información que pueda ayudar en el proceso de toma de decisiones.

Los modelos de aprendizaje automático están ayudando completamente en el desempeño de estas campañas. 

## Dataset

Este conjunto de datos está relacionado con las campañas de marketing directo de una institución bancaria portuguesa. Las campañas de marketing se basaron en llamadas telefónicas. A menudo, se requería más de un contacto con el mismo cliente, para poder acceder a si el producto (depósito bancario a plazo) estaría ('sí') o no ('no') suscrito.

El objetivo es predecir si el cliente se suscribirá (sí/no) a un depósito a plazo, construyendo un modelo de clasificación utilizando árboles de decisión.

## Summay of data
### Categorical Variables :
job : admin,technician, services, management, retired, blue-collar, unemployed, entrepreneur, housemaid, unknown, self-employed, student

marital : married, single, divorced

education: secondary, tertiary, primary, unknown

default : yes, no

housing : yes, no

loan : yes, no

deposit : yes, no (Dependent Variable)

contact : unknown, cellular, telephone

month : jan, feb, mar, apr, may, jun, jul, aug, sep, oct, nov, dec

poutcome: unknown, other, failure, success


### Numerical Variables:
age

balance

day

duration

campaign -> numero de veces que se le ha contactado en esta campaña

pdays -> dias que han pasado desde el último contacto, -1 si no se le ha contactado

previous -> numero de veces que se le ha contactado previas a la campaña

In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn import datasets
from io import StringIO
from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import metrics
%matplotlib inline

In [12]:
# Crear dataframe a partir de .csv
df = pd.read_csv('https://raw.githubusercontent.com/manjam/Saturday4AI/main/bank.csv',error_bad_lines=False)

# Mostrar número de filas y columnas del dataframe
print("Filas: " + str(df.shape[0]) + " Cols: " + str(df.shape[1]))

# Mostrar las primeras 10 filas (TO-DO)
df.head(10)

Filas: 11162 Cols: 17


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes
5,42,management,single,tertiary,no,0,yes,yes,unknown,5,may,562,2,-1,0,unknown,yes
6,56,management,married,tertiary,no,830,yes,yes,unknown,6,may,1201,1,-1,0,unknown,yes
7,60,retired,divorced,secondary,no,545,yes,no,unknown,6,may,1030,1,-1,0,unknown,yes
8,37,technician,married,secondary,no,1,yes,no,unknown,6,may,608,1,-1,0,unknown,yes
9,28,services,single,secondary,no,5090,yes,no,unknown,6,may,1297,3,-1,0,unknown,yes


In [13]:
# Hallar número de valores únicos en cada columna (TO-DO)
for col in df:
  print(len(df[col].unique()))

76
12
3
4
2
3805
2
2
3
31
12
1428
36
472
34
4
2


In [14]:
# Comprobar la existencia de valores nulos en el dataset
df.isnull().values.any()


False

In [15]:
# Mostrar información general del dataframe
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   job        11162 non-null  object
 2   marital    11162 non-null  object
 3   education  11162 non-null  object
 4   default    11162 non-null  object
 5   balance    11162 non-null  int64 
 6   housing    11162 non-null  object
 7   loan       11162 non-null  object
 8   contact    11162 non-null  object
 9   day        11162 non-null  int64 
 10  month      11162 non-null  object
 11  duration   11162 non-null  int64 
 12  campaign   11162 non-null  int64 
 13  pdays      11162 non-null  int64 
 14  previous   11162 non-null  int64 
 15  poutcome   11162 non-null  object
 16  deposit    11162 non-null  object
dtypes: int64(7), object(10)
memory usage: 1.4+ MB


In [16]:
# Descripción analítica básica del dataframe
df.describe()


,age,balance,day,duration,campaign,pdays,previous
count,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000
mean,41.231948,1528.538524,15.658036,371.993818,2.508421,51.330407,0.832557
std,11.913369,3225.413326,8.420740,347.128386,2.722077,108.758282,2.292007
min,18.000000,-6847.000000,1.000000,2.000000,1.000000,-1.000000,0.000000
25%,32.000000,122.000000,8.000000,138.000000,1.000000,-1.000000,0.000000
50%,39.000000,550.000000,15.000000,255.000000,2.000000,-1.000000,0.000000
75%,49.000000,1708.000000,22.000000,496.000000,3.000000,20.750000,1.000000
max,95.000000,81204.000000,31.000000,3881.000000,63.000000,854.000000,58.000000


In [17]:
# Distribución de la Edad
df['age'].describe()

count    11162.000000
mean        41.231948
std         11.913369
min         18.000000
25%         32.000000
50%         39.000000
75%         49.000000
max         95.000000
Name: age, dtype: float64

### Convertir datos categóricos si hay y hacer una copia de los datos


In [29]:
# Hacer una copia primero!!
bank_data = df.copy()
col_names = list(bank_data)
categories = list(['job','marital','default','education','housing','loan','contact','month','poutcome','deposit'])
print("\nNames of dataframe columns")
print(col_names)
print(categories)

# loop to change each column to category type
for col in col_names:
 if col in categories:
    bank_data[col] = bank_data[col].astype('category')
bank_data.info()



Names of dataframe columns
['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'deposit']
['job', 'marital', 'default', 'education', 'housing', 'loan', 'contact', 'month', 'poutcome', 'deposit']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   age        11162 non-null  int64   
 1   job        11162 non-null  category
 2   marital    11162 non-null  category
 3   education  11162 non-null  category
 4   default    11162 non-null  category
 5   balance    11162 non-null  int64   
 6   housing    11162 non-null  category
 7   loan       11162 non-null  category
 8   contact    11162 non-null  category
 9   day        11162 non-null  int64   
 10  month      11162 non-null  category
 11  duration   11162 non-null  int64   
 12  camp

#### Job

In [37]:
# Explorar Personas que hicieron un depósito Vs Categoría de trabajo

jobs=bank_data['job'].unique()
#type(jobs)
#print(jobs)
for j in jobs:
    print("{:15} : {:5}". format(j, len(bank_data[(bank_data.deposit == "yes") & (bank_data.job ==j)])))

admin.          :   631
technician      :   840
services        :   369
management      :  1301
retired         :   516
blue-collar     :   708
unemployed      :   202
entrepreneur    :   123
housemaid       :   109
unknown         :    34
self-employed   :   187
student         :   269


In [40]:
# Imprime los diferentes tipos de categorías de trabajo y cuántos elementos hay en cada uno (investiga value_counts)
bank_data.job.value_counts()


management       2566
blue-collar      1944
technician       1823
admin.           1334
services          923
retired           778
self-employed     405
student           360
unemployed        357
entrepreneur      328
housemaid         274
unknown            70
Name: job, dtype: int64

In [45]:
# Combinar trabajos similares en categorías. Para ello investigad la función replace
# ['management', 'admin.'] ->  'white-collar'
# ['services','housemaid'] -> 'pink-collar'
# ['retired', 'student', 'unemployed', 'unknown'] ->  'other'

bank_data=bank_data.replace(to_replace=['management','admin'],value='white-collar')
bank_data=bank_data.replace(to_replace=['services','housemaid'],value='pink-collar')
bank_data=bank_data.replace(to_replace=['retired','student','unemployed','unknown'],value='other')

jobs=bank_data['job'].unique()
for j in jobs:
    print("{:15} : {:5}". format(j, len(bank_data[(bank_data.deposit == "yes") & (bank_data.job ==j)])))

admin.          :   631
technician      :   840
pink-collar     :   478
white-collar    :  1301
other           :  1021
blue-collar     :   708
entrepreneur    :   123
self-employed   :   187


In [46]:
# Volved a hacer el conteo de valores para ver como ha quedado la información
bank_data.job.value_counts()

white-collar     2566
blue-collar      1944
technician       1823
other            1565
admin.           1334
pink-collar      1197
self-employed     405
entrepreneur      328
Name: job, dtype: int64

#### poutcome

In [52]:
# Exploremos los valores que tiene
bank_data.poutcome.value_counts()
#bank_data.poutcome.describe()

other      8863
failure    1228
success    1071
Name: poutcome, dtype: int64

In [54]:
# Combina "unknown" y "other" ya que "other" no es realmente compatible con el "success" o el "failure". Reutiliza la función replace
bank_data['poutcome'] = bank_data['poutcome'].replace(to_replace=['other'],value='unknown')
bank_data.poutcome.value_counts()

unknown    8863
failure    1228
success    1071
Name: poutcome, dtype: int64

#### contact

In [55]:
# Drop 'contact' ya que no nos aporta información
#bank_data.drop('contact')
bank_data.info()

KeyError: ignored

#### default

In [ ]:
# Valores de "default" : Cambiar a  yes/no
bank_data["default"]
bank_data['default_cat'] = bank_data['default'].map( {'yes':1, 'no':0} )
bank_data.drop('default', axis=1,inplace = True)

#### housing, loan, deposit

Ahora repetid lo mismo con estas variables!


In [ ]:
# values for "housing" : yes/no


In [ ]:
# values for "loan" : yes/no


In [ ]:
# values for "deposit" : yes/no


In [ ]:
# pdays: número de días que pasaron después de que el cliente fue contactado por última vez de una campaña anterior
# -1 significa que el cliente no fue contactado previamente

print("Customers that have not been contacted before:", len(bank_data[bank_data.pdays==-1]))
print("Maximum values on padys    :", bank_data['pdays'].max())

In [ ]:
# sustituye el valor -1 de pdays por un valor grande i.e. 10000 para que refleje que hace mucho que no se le ha contactado (similar a no haberse contatado nunca)



In [ ]:
# Cread una nueva columna que contenga recent_pdays, queriendo que reefleje aquellos que son recientes como los valores mas grandes y los que nunca se les ha contactado
# los valores más pequeños (siempre positivos). Para esto, haced el inverso del valor de pdays. Una vez hecho, eliminad pdays
# Create a new column: recent_pdays 


### Convert to dummy values

In [ ]:
# Convertid las variables a dummies: Es decir, para que sea tratable como vectores, es decir, números, requerimos cambiar las variables categóricas a numéricas.
# Para ello, lo que se hace es, para cada variable categórica, sustituirla por tantas variables como  valores tenga la columna, que contengan 1 si esa fila tenia ese valor y 0 si no
# investiga get_dummies
bank_with_dummies = 

In [ ]:
bank_with_dummies.shape


In [ ]:
bank_with_dummies.describe()


In [ ]:
# Scatterplot con la edad y el dinero que hay en la cuenta (balance). Puedes hacerlo con pandas y su función plot o con scatterplot de seaborn


# ¿Qué interpretas?

In [ ]:
# Histograma con la variable poutcome_success



In [ ]:
# Personas que se inscriben en un depósito a plazo
bank_with_dummies[bank_data.deposit_cat == 1].describe()


In [ ]:
# Diagrama de barras mostrando el valor de desposite separado por trabajo. Investigad barplot de seaborn
plt.figure(figsize = (10,6))


### Establecer las relaciones entre las features


In [ ]:
# Mostrar matriz de correlación de variables
# Pista: Explora las funciones corr de pandas y heatmap de seaborn

In [ ]:
# Mostrar correlaciones como una función discreta entre las diferentes variables con una matriz
# útil para apreciar relaciones lineales

# Pista: explore pd.plotting.scatter_matrix

In [ ]:
# Partir el test en cierta proporción (¡experimentar!) Importa la libreria sklearn y utiliza train test split ;)


In [ ]:
# Definir un clasificador
from sklearn.neighbors import KNeighborsClassifier

k = #...

KNeighborsClassifier(n_neighbors=k)
# Entrenar el clasificador con el dataset de train
neigh.fit(X, y)
# Predecir valores para las variables independientes de test
neigh.predict(# los datos de test)
# Calcular la precisión
neigh.score(#los datos de test X, los datos de test y)

